# Imports

Libraries

In [121]:
# Basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch libraries
import torch

#Neural network libraries
import torch.nn as nn
import torch.nn.functional as F

# #Creating optimizer
import torch.optim as optim

# For terminating program (useful for stopping before training is complete and seeing entire result)
import signal
import sys, os

CSV File

In [122]:
script_dir = os.path.abspath('') # absolute file path in Jupyter
file_path = os.path.join(script_dir, "AI EarthHack Dataset.csv") # file path for the text file input

dataset = pd.read_csv(file_path, header=None)
print(dataset[1])

0                                                 problem
1       The construction industry is indubitably one o...
2       I'm sure you, like me, are feeling the heat - ...
3       The massive shift in student learning towards ...
4       The fashion industry is one of the top contrib...
                              ...                        
1296    The linear 'take, make, dispose' model of prod...
1297    The conundrum we face is the improper disposal...
1298               This solution will help the vegetation
1299    Accumulation and improper disposal of single-u...
1300    The excessive and wasteful resource consumptio...
Name: 1, Length: 1301, dtype: object


# Zero-Shot Classification

In [131]:
from transformers import pipeline

# TODO look into changing the modle, bart-large-mnli seems to be the most populat for ZSC
# Other options are T5, GPT, and RoBERTa

# Models tried:
# - facebook/bart-large-mnli
# - sjrhuschlee/flan-t5-base-mnli
# - google/flan-t5-base

classifier = pipeline("zero-shot-classification", model="sjrhuschlee/flan-t5-base-mnli")

# Example class descriptions and input text
# input_text = [["Problem: "+str(dataset[1][i]), "Solution: "+str(dataset[2][i])] for i in range(1,3)]
# Bad solutions: 39, 52, 22, 457, 279
# Good solutions: 3, 117

class_descriptions1=[
    """
    The problem and solution are detailed, specific, and in-depth.
    Bonus points if they are related to circular economy.
    """,
    """
    The problem and/or solution are sloppy, off-topic (i.e., not sustainability related), unsuitable, or vague (such as the over-generic content that prioritizes form over substance, offering generalities instead of specific details).
    In addition, inputs such as, 'Problem: I will research to make solution Solution: Hi' belong in this category
    """
]

class_descriptions2=[
    """
    The solution is feasible.
    """,
    """
    The solution is not feasible.
    This can mean that the solution is too wide in scope to be easily implemented, or puts unreasonable expectations on people, businesses, or the world as a whole.
    Any solution that would take a large amount of resources or manpower to execute successfully would also be considered not feasible.
    """
]

class_descriptions3=[
    """
    The solution is innovative.
    A solution is innovative if it imagines something completely new and novel.
    """,
    """
    The solution is not innovative.
    A solution is considered not innovative if it does not present any originality.
    """
]

class_descriptions4 = [
    """
    The solution is scalable.
    A scalable solution will often provide a clear path on how the solution can be scaled in the future, if required.
    A scalable solution is one that can start small, and grows to have a larger area of effect over time.
    """,
    """
    The solution is not scalable.
    These solutions are ones that only focus on the small scale, and could not easily be done on a large scale.
    """
]

class_descriptions5 = [
    """
    The solution is not pertinent to a circular economy.
    In today’s rapidly evolving world, climate change stands as a formidable problem, deeply influencing our daily lives and the health of our planet. The circular economy, with its focus on reusing and recycling resources to minimize waste, emerges as a crucial strategy in this battle. Innovations like car-sharing platforms significantly reduce the carbon footprint of transportation, while modular designs in various products promote waste reduction by allowing individual components to be upgraded rather than discarding the entire item.
    In the face of climate change's criticality, the urgency to identify and implement high-impact circular economy solutions has never been greater. The challenge we confront today, however, extends beyond coming up with solutions to confront this problem. It lies in the daunting task of effectively evaluating a vast and diverse array of solutions, discerning the most impactful ones amidst a sea of possibilities. This process can be overwhelming, given the complexity and the sheer volume of potential solutions, leading to cognitive overload for human evaluators.
    """,
    "The solution is not pertinent to a circular economy."
]

all_descriptions = {
    "Relevance": class_descriptions1,
    "Feasibility": class_descriptions2,
    "Innovation": class_descriptions3,
    "Scalability": class_descriptions4,
    "Circular Economic": class_descriptions4
}

scores = {
    "Relevance": [],
    "Feasibility": [],
    "Innovation": [],
    "Scalability": [],
    "Circular Economic": []
}

# Scalability 40

low_limit = 1
up_limit = 10

for i in range(1,up_limit+1):
    input_text = "Problem: " + str(dataset[1][i]) + " Solution: " + str(dataset[2][i]).replace("\"\"\"\"", "")
    for category in all_descriptions:
    #category = "Relevance"
        result = classifier(input_text, all_descriptions[category])
        for label,score in zip(result["labels"], result["scores"]):
            if label==all_descriptions[category][0]:
                scores[category].append(score)
                # print(dataset[0][i] + ": " + str(scores[category][(j := j+1)]))

print(scores)

{'Relevance': [0.9193720817565918, 0.9271174669265747, 0.9306560754776001, 0.8955116271972656, 0.9487287998199463, 0.9185700416564941, 0.5345593094825745, 0.9396084547042847, 0.9541129469871521, 0.7660059928894043], 'Feasibility': [0.9248408079147339, 0.819028913974762, 0.9659156799316406, 0.7874659895896912, 0.9630635380744934, 0.9665031433105469, 0.42798101902008057, 0.9326707720756531, 0.943945586681366, 0.904151439666748], 'Innovation': [0.8587149381637573, 0.9497579336166382, 0.7300000190734863, 0.7523269057273865, 0.9739944338798523, 0.6928094625473022, 0.24945354461669922, 0.9613739848136902, 0.8725293874740601, 0.3659786880016327], 'Scalability': [0.9431308507919312, 0.5450352430343628, 0.8402358293533325, 0.6722171306610107, 0.9486923813819885, 0.9851491451263428, 0.3123508095741272, 0.8632518649101257, 0.9745986461639404, 0.6450303196907043], 'Circular Economic': [0.9431308507919312, 0.5450352430343628, 0.8402358293533325, 0.6722171306610107, 0.9486923813819885, 0.98514914512

In [130]:
# Putting in another CSV
obj1 = {'Id':dataset[0][low_limit:up_limit+1], 'Problem':dataset[1][low_limit:up_limit+1], 'Solution':dataset[2][low_limit:up_limit+1]}
obj1.update(scores)
df1 = pd.DataFrame(obj1)

new_csv = 'Analyzed_dataset.csv'
script_dir = os.path.abspath('')
file_path = os.path.join(script_dir,new_csv)
df1.to_csv(file_path,index=False)